In [12]:
!pip install transformers accelerate --upgrade

In [13]:
!pip install beautifulsoup4 requests

In [ ]:
!huggingface-cli login --token {"YOR_TOKEN"}

In [15]:
# ✅ وارد کردن کتابخانه‌ها
import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig
import torch

In [16]:
DEVICE = "cpu"
if torch.backends.mps.is_available():
    DEVICE = "mps"
elif torch.cuda.is_available():
    DEVICE = "cuda"

print(f"Using device: {DEVICE}")

Using device: cuda


In [17]:
# نام مدل
model_id = "meta-llama/Llama-3.2-1B-Instruct"

# توکنایزر و مدل را بارگذاری می‌کنیم
tokenizer = AutoTokenizer.from_pretrained(model_id, use_auth_token=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,  # استفاده از float16 برای GPU
    device_map="auto"           # به صورت خودکار روی GPU یا CPU قرار می‌گیره
)
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/tokenization_auto.py:897: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


In [18]:
def generate_response(prompt, model, tokenizer, **generation_kwargs):
    """
    Generates a response from the model given a prompt string.
    """
    gen_config = getattr(model, "generation_config", GenerationConfig())

    default_generation_kwargs = {
        "do_sample": gen_config.do_sample,
        "top_p": gen_config.top_p,
        "temperature": gen_config.temperature,
        "max_new_tokens": 256
    }

    generation_kwargs = {**default_generation_kwargs, **generation_kwargs}

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        **generation_kwargs
    )

    input_length = inputs["input_ids"].shape[1]
    return tokenizer.decode(outputs[0][input_length:], skip_special_tokens=True)


In [19]:
def format_messages_as_prompt(messages, tokenizer):
    """
    Converts a list of chat-style messages (with roles) into a formatted prompt
    using the tokenizer's chat template (e.g., for LLaMA 3 chat models).
    """
    return tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

In [20]:
# ✅ کلاس Webpage برای گرفتن و تمیز کردن متن وب‌سایت
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [21]:
# Let's try one out. Change the website and add print statements to follow along.

ed = Website("https://edwarddonner.com")
print(ed.title)
print(ed.text)

Home - Edward Donner
Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.
We work with groundbreaking, proprietary LLMs verticalized for talent, we’ve
patented
our matching model, and our award-winning platform has happy customers and tons of press coverage.
Connec

In [22]:
# ✅ پیام‌ها با نقش‌های مشخص
messages = [
    {
        "role": "system",
        "content": "You are an assistant that summarizes websites. Respond in markdown, and ignore any navigation text."
    },
    {
        "role": "user",
        "content": f"You are looking at a website titled: {ed.title}\n\nThe contents of this website is as follows:\n\n{ed.text}"
    }
]

# ✅ تبدیل پیام‌ها به prompt نهایی با قالب chat
prompt = format_messages_as_prompt(messages, tokenizer)

# ✅ تولید پاسخ با مدل
summary = generate_response(prompt, model, tokenizer, temperature=0.7)

# ✅ نمایش خروجی خلاصه‌شده
from IPython.display import Markdown, display
display(Markdown(summary))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


# Home
### Overview of Edward Donner's Website

Edward Donner's website is a personal and professional portfolio showcasing his work in artificial intelligence, particularly in the realm of Large Language Models (LLMs). The website highlights his involvement with Nebula.io, a company applying AI to talent discovery and management.

### About
Edward Donner is the co-founder and CTO of Nebula.io, a company that utilizes LLMs to source, understand, engage, and manage talent. He previously founded and served as CEO of AI startup untapt, which was acquired in 2021.

### Connect
For more information, connect with Edward on various platforms:

* LinkedIn
* Twitter
* Facebook

### Resources
A collection of resources on AI and LLM engineering, including:

* **January 23, 2025:** LLM Workshop – Hands-on with Agents
* **December 21, 2024:** Mastering AI and LLM Engineering
* **October 16, 2024:** From Software Engineer to AI Data Scientist